In [1]:

import pandas as pd
from tqdm import tqdm

In [7]:
a = pd.read_json('EventNarrative/Instances/EN_test.json')
b = pd.read_json('EventNarrative/Instances/EN_train.json')
c = pd.read_json('EventNarrative/Instances/EN_validation.json')

In [15]:
#read json files without pandas
import json
with open('EventNarrative/Instances/EN_test.json') as test:
    test = json.load(test)
    test = test[:int(len(test)/4)]
with open('EventNarrative/Instances/EN_train.json') as train:  
    train = json.load(train)
    train = train[:int(len(train)/4)]
with open('EventNarrative/Instances/EN_validation.json') as val:      
    val = json.load(val)
    val = val[:int(len(val)/4)]




In [11]:
len(test)
len(train)



44886

In [22]:
with open('EventNarrative/short_Instances/EN_test.json', 'w',  encoding='utf-8') as f:
    # write the dictionary to the file as JSON
    json.dump(test, f, indent = 4)
with open('EventNarrative/short_Instances/EN_train.json', 'w',  encoding='utf-8') as f:
    # write the dictionary to the file as JSON
    json.dump(train, f, indent = 4)
with open('EventNarrative/short_Instances/EN_validation.json', 'w',  encoding='utf-8') as f:
    # write the dictionary to the file as JSON
    json.dump(val, f, indent = 4)

In [23]:
from distutils.command import check


def check_nonASCI(file):
    with open(file, 'r') as f:
        data = json.load(f)

    for value in data:
        if isinstance(value, str):
            for character in value:
                if ord(character) > 127:
                    print("JSON file contains non-ASCII character(s)")
                    break
            else:
                continue
            break
    else:
        print("JSON file does not contain non-ASCII characters")

check_nonASCI('EventNarrative/short_Instances/EN_test.json')
check_nonASCI('EventNarrative/short_Instances/EN_train.json')
check_nonASCI('EventNarrative/short_Instances/EN_validation.json')       

JSON file does not contain non-ASCII characters
JSON file does not contain non-ASCII characters
JSON file does not contain non-ASCII characters


In [14]:
from datasets import load_dataset, load_metric
#dataset = load_dataset('json', data_files={'train': train[:10000]  , 'valid': val[:1000], 'test': test[:1000]})
train_file, test_file, dev_file = 'EventNarrative/Instances/EN_train.json', 'EventNarrative/Instances/EN_test.json', 'EventNarrative/Instances/EN_validation.json'

dataset = load_dataset('json', data_files={'train': train_file, 'valid': dev_file, 'test': test_file})

Using custom data configuration default-40154fb1efae9a5b


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 454.90it/s]


Dataset json downloaded and prepared to /Users/macoftraopia/.cache/huggingface/datasets/json/default-40154fb1efae9a5b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 103.83it/s]


In [26]:
a = dataset['train'][:10000]


AttributeError: 'dict' object has no attribute 'num_rows'

In [29]:
dataset['train'] = dataset['train'][:10000]
dataset['valid'] = dataset['valid'][:1000]
dataset['test'] = dataset['test'][:1000]

{'Instance_Knowledge_Graph': ['Jamaica at the FIFA World Cup - sport - association football | Jamaica at the FIFA World Cup - subclass of - Jamaica national football team | story - hasCore - Jamaica at the FIFA World Cup | story - type - FIFA World Cup team',
  '1906 Swarthmore Garnet Tide football team - point in time - 1906 | story - hasCore - 1906 Swarthmore Garnet Tide football team | story - type - American football team season',
  'Georgia–Holy See relations - country - Georgia | Georgia–Holy See relations - country - Vatican City | story - hasCore - Georgia–Holy See relations | story - type - bilateral relation',
  '2007 Rhythmic Gymnastics European Championships - point in time - 2007 | 2007 Rhythmic Gymnastics European Championships - location - Baku | story - hasCore - 2007 Rhythmic Gymnastics European Championships | story - type - sports season',
  "1958 Formula One season - winner - Vanwall | 1958 Formula One season - winner - Mike Hawthorn | 1958 Formula One season - spor

## Preprocess EventNarrative
- Map entities to text
- Linearize KG according to format
- Save in Csv format


#### Next steps:
- NER to create EventKG (need Actors, locations, times - topic and titles already there)




In [46]:
def preprocess_data(path_in, path_out):
    data = pd.read_json(path_in)
    for instance in tqdm(range(len(data))):
        for key,value in data['entity_ref_dict'][instance].items():
            data['narration'][instance] = data['narration'][instance].replace(key,value)
        formatted_triples = []
        for triple in data['keep_triples'][instance]: 
            #formatted_triples += (" | ".join([" - ".join(triple) for triple in data['keep_triples'][instance]]))
            formatted_triples.append((" | ".join([" - ".join(triple) for triple in data['keep_triples'][instance]])))
            data['keep_triples'][instance] = formatted_triples[0]
    # data['types'] = [eval(data['types'][i]) for i in range(len(data['types']))]
    # def concatenate_list(lst):
    #     return ', '.join(lst)
    # result = pd.DataFrame()
    # result['Instance Knowledge Graph'] = data['keep_triples'] + ' | Event - title - ' + data['Event_Name'] + ' | Event - topic - ' +  data['types'].apply(concatenate_list).apply(lambda x: x.replace(',' , ' | Event - topic - '))
    # result['story'] = data['narration']         
    # result.to_csv(path_out, index=False)
    df2 = data.loc[:, ['Event_Name', 'narration', 'keep_triples', 'types']]
    df2.to_csv(path_out, index=False)

In [5]:
test = pd.read_csv('EventNarrative/test_data.csv')
test.head(3)

,Event_Name,narration,keep_triples,types
0,1953 Cannes Film Festival,The 1953 Cannes Film Festival was held from 15...,1953 Cannes Film Festival - point in time - 1953,['film festival edition']
1,1988 Benson and Hedges Open,The 1988 Benson and Hedges Open was a men's 19...,1988 Benson and Hedges Open - located in the a...,"['Heineken Open', 'tennis tournament edition']"
2,1992 King Fahd Cup knockout stage,The 1992 King Fahd Cup knockout stage began on...,1992 King Fahd Cup knockout stage - point in t...,"['knockout stage', 'sports season']"


In [16]:
#eval(test['types'][1])
#test['types'][1] = eval(test['types'][1])
test['types'][1]

['Heineken Open', 'tennis tournament edition']

In [47]:
#preprocess_data('EventNarrative/test_data.json', 'EventNarrative/test_data.csv')
#preprocess_data('EventNarrative/dev_data.json', 'EventNarrative/dev_data.csv')
preprocess_data('EventNarrative/train_data.json', 'EventNarrative/train_data.csv')

100%|██████████| 179544/179544 [1:15:18<00:00, 39.73it/s]


In [48]:
test = pd.read_csv('EventNarrative/train_data.csv')
test['types'] = [eval(test['types'][i]) for i in range(len(test['types']))]
def concatenate_list(lst):
    return ', '.join(lst)
result = pd.DataFrame()
result['Instance Knowledge Graph'] = test['keep_triples'] + ' | Event - title - ' + test['Event_Name'] + ' | Event - topic - ' +  test['types'].apply(concatenate_list).apply(lambda x: x.replace(',' , ' | Event - topic - '))
result['story'] = test['narration'] 
#result = result.drop(columns=["Unnamed: 0"])
result.to_csv('EventNarrative/Instance_train.csv', index=False)

In [19]:
def concatenate_list(lst):
    return ', '.join(lst)

def preprocess_data(path_in, path_out):
    data = pd.read_json(path_in)
    for instance in tqdm(range(len(data))):
        if type(data['types'][instance]) != list:
            data['types'][instance]= eval(data['types'][instance])
        for key,value in data['entity_ref_dict'][instance].items():
            data['narration'][instance] = data['narration'][instance].replace(key,value)
        formatted_triples = []
        for triple in data['keep_triples'][instance]: 
            formatted_triples.append((" | ".join([" - ".join(triple) for triple in data['keep_triples'][instance]])))
            data['keep_triples'][instance] = formatted_triples[0]
        
    #data['types'] = [eval(data['types'][i]) for i in range(len(data['types']))]    

    result = pd.DataFrame()  
    result['Instance Knowledge Graph'] = data['keep_triples'] + ' | story - hasCore - ' + data['Event_Name'] + ' | story - type - ' +  data['types'].apply(concatenate_list).apply(lambda x: x.replace(',' , ' | story - type - '))
    result['story'] = data['narration'] 
    result.to_csv(path_out, index=False)

preprocess_data('EventNarrative/test_data.json', 'EventNarrative/test_data.csv')    

100%|██████████| 22442/22442 [00:59<00:00, 374.57it/s]


In [15]:
result.head(3)

,Instance Knowledge Graph,story
0,1953 Cannes Film Festival - point in time - 19...,The 1953 Cannes Film Festival was held from 15...
1,1988 Benson and Hedges Open - located in the a...,The 1988 Benson and Hedges Open was a men's 19...
2,1992 King Fahd Cup knockout stage - point in t...,The 1992 King Fahd Cup knockout stage began on...


In [24]:
result['Instance Knowledge Graph'][0]

'Brunei at the 2013 World Aquatics Championships - point in time - 2013 | Brunei at the 2013 World Aquatics Championships - participant in - 2013 World Aquatics Championships | Brunei at the 2013 World Aquatics Championships - location - Spain | Brunei at the 2013 World Aquatics Championships - location - Barcelona | Brunei at the 2013 World Aquatics Championships - participant of - 2013 World Aquatics Championships | Event - title - Brunei at the 2013 World Aquatics Championships | Event - topic - nation at sport competition'

In [42]:
import csv
import json

csv_filename = "EventNarrative/Instance_train.csv"
#json_filename = "EventNarrative/Instances/EN_validation.json"
json_filename = "EventNarrative/Instances/EN_train.json"



with open(csv_filename, newline='') as csvfile:
    reader = csv.reader(csvfile)
    columns = next(reader)

# Read the data from the CSV file and write it to a JSON file
with open(json_filename, "w") as jsonfile:
    jsonfile.write('[')
    for row in csv.DictReader(open(csv_filename), fieldnames=columns):
        json.dump(row, jsonfile, indent = 4)
        jsonfile.write(',')
        jsonfile.write('\n')
    jsonfile.write('{}')    
    jsonfile.write(']')    

In [ ]:
def convert_json(input_file, output_file):
    with open(output_file, "w") as jsonfile:
        for row in csv.DictReader(open(input_file), fieldnames=columns):
            json.dump(row, jsonfile, indent = 4)
            jsonfile.write(',')
            jsonfile.write('\n')
        

